In [1]:
import pandas as pd
from pathlib import Path
from sklearn.cluster import DBSCAN, HDBSCAN
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import osmnx as ox
from shapely.geometry import Polygon
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
import warnings
from scipy.spatial.distance import cdist
import plotly.express as px
import plotly.graph_objects as go
from scipy.spatial.distance import pdist, squareform

token = "pk.eyJ1IjoicGFibG9nYXNjbyIsImEiOiJjbHN0MHloeGwweGwzMmtxaXFobDNwc29tIn0.cXi2ESWBKoAPywLexpqruQ"

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'matplotlib'

In [10]:
from shapely.geometry import Polygon
polygon = Polygon([(0, 0), (0, 2), (2, 2), (2,0)])
polygon2 = Polygon([(0, 0), (0, 2), (2, 2), (2, 0)])
polygon3= polygon2
polygon2

In [2]:
def calculate_distance(df):
    # Radius of the Earth in kilometers
    r = 6371.0

    # Convert degrees to radians
    lat1 = np.radians(df['latitude'])
    lon1 = np.radians(df['longitude'])
    lat2 = np.radians(df['home_lat'])
    lon2 = np.radians(df['home_lon'])

    # Differences
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Distance in kilometers between the points on the sphere
    distance_sphere = r * c

    # Difference in elevation between the points
    delev = df['elevation']

    # Euclidean distance
    distance = np.sqrt(distance_sphere**2 + delev**2)

    return distance

In [ ]:
df.rename(columns={"journey":"test"}, inplace=True)
df.head()

,time,test,ident,model,source,location,timestamp,latitude,longitude,altitude,...,rssi,home_lat,home_lon,home_height,speed,heading,station_name,station_latitude,station_longitude,distance
0,1696140523,36186,3YTBKBF00402PS,Mavic Air 2S,as,1671673,2023-10-01 06:08:43,50.848148,4.350050,157.0,...,NaN,50.848343,4.350222,126.0,9.0,302.0,0QRDKC2R03J32P,50.883286,4.428613,27.000011
8,1696141894,36189,163CGCKR0A6S70,Mavic 2,as,1671720,2023-10-01 06:31:34,50.873692,4.470028,111.0,...,NaN,50.871891,4.469329,47.0,32.0,359.0,0QRDKC2R03J32P,50.883293,4.428593,55.000386
9,1696141898,36189,163CGCKR0A6S70,Mavic 2,as,1671721,2023-10-01 06:31:38,50.874050,4.470022,111.0,...,NaN,50.871891,4.469329,47.0,22.0,359.0,0QRDKC2R03J32P,50.883293,4.428592,57.000526
10,1696141900,36189,163CGCKR0A6S70,Mavic 2,as,1671722,2023-10-01 06:31:40,50.874096,4.470022,111.0,...,NaN,50.871891,4.469329,47.0,0.0,359.0,0QRDKC2R03J32P,50.883293,4.428592,57.000548
11,1696141902,36189,163CGCKR0A6S70,Mavic 2,as,1671723,2023-10-01 06:31:42,50.874096,4.470022,111.0,...,NaN,50.871891,4.469329,47.0,0.0,277.0,0QRDKC2R03J32P,50.883293,4.428592,57.000548


In [ ]:
# Load the dataframe outside the function
data_dir = Path("/data/pgcasado/projects/ACUTE/data/external")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )
#'2023-06-21'
mask = (df['timestamp'] > '2023-10-01') & (df['timestamp'] <= '2023-12-31') & (df["station_name"] == "0QRDKC2R03J32P") & (df["latitude"] > 3)
df = df[mask] # Create a copy to avoid SettingWithCopyWarning
df['distance'] = df.apply(calculate_distance, axis=1)

In [ ]:
cluster_df = df.groupby('journey').agg({
            'altitude': ['count', 'max', 'std', 'mean'],
            'distance': ['max', 'std', 'mean', lambda x:x],
        })
[' '.join(col).strip() for col in cluster_df.columns.values]

['altitude count',
 'altitude max',
 'altitude std',
 'altitude mean',
 'distance max',
 'distance std',
 'distance mean',
 'distance <lambda_0>']

In [ ]:
# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

grouped_df = df.groupby("journey").agg(
    latitude=("latitude", "mean"),
    longitude=("longitude", "mean"),
    altitude=("altitude", "max"),
    distance=("distance", "max"),
    ident = ("ident", "first"),
    model = ("model", "first"),
    timestamp = ("timestamp", "first"),
)

for index, row in grouped_df.iterrows():
    centroid_coords = row[["latitude", "longitude"]].values.reshape(1, -1).astype(float)
    points = df[["latitude", "longitude"]].values.astype(float)
    distances = cdist(centroid_coords, points, metric="euclidean")
    # Find the maximum distance for each centroid
    max_distance = distances.max(axis=1)
    grouped_df.loc[index, "radius"] = max_distance
   
df = grouped_df.copy() 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

data_dir = Path("/data/pgcasado/datasets/acute/")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )
#'2023-06-21'
mask = (df['timestamp'] > '2023-10-01') & (df['timestamp'] <= '2023-12-31') & (df["station_name"] == "0QRDKC2R03J32P") & (df["latitude"] > 3)
df = df[mask] # Create a copy to avoid SettingWithCopyWarning
df['distance'] = df.apply(calculate_distance, axis=1)

coordinates_df = df[['latitude', 'longitude']]
z_scores = np.abs((coordinates_df - coordinates_df.mean()) / coordinates_df.std())
threshold = 1
mask = (z_scores < threshold).all(axis=1)
df = df[mask]
mask_alt = (df["altitude"] > 100)
df_filtered_mask = df.loc[mask_alt].copy()

# Define your latitude and longitude limits
lat_min, lat_max = df_filtered_mask["latitude"].min(), df_filtered_mask["latitude"].max()
lon_min, lon_max = df_filtered_mask["longitude"].min(), df_filtered_mask["longitude"].max()
print(lat_min, lat_max, lon_min, lon_max)
# Create a square polygon
polygon = Polygon([(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)])

#tags = {"building":["university", "chapel"], "amenity" : ["prison",]}
tags = {"building":["university",]}
dfx = ox.features.features_from_polygon(polygon, tags= tags).copy()  # Create a copy to avoid SettingWithCopyWarning
dfx.dropna(subset=['name'], inplace=True)
dfx['longitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
dfx['latitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])

50.83266067504883 50.942623138427734 4.357229232788086 4.513183116912842


/data/pgcasado/envs/acute/lib/python3.12/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/data/pgcasado/envs/acute/lib/python3.12/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
/data/pgcasado/envs/acute/lib/python3.12/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/data/pgcasado/envs/acute/lib/python3.12/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/data/pgcasado/envs/acute/lib/python3.12/site-pa

In [ ]:
dfx["name"]

/data/pgcasado/envs/acute/lib/python3.12/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)


element_type  osmid    
way           138300502                                             Arlon 14
              228640482                                       Centre Faculté
              228640486                                              Ehrlich
              228640489                                    Rosalind Franklin
              228640491                                  Auditoires centraux
              228640493                                               Galien
              228640497                                          Van Helmont
              228640499                                               Vésale
              228641523                                       Claude Bernard
              228641530                                                  ICP
              228683675                                    Centre œcuménique
              228686106                                              La Mémé
              228686346                             

In [ ]:
import geopy
geo = geopy.geocoders.Nominatim(user_agent="test")
geo.geocode("Tintin Comic Mural", exactly_one=False)

In [ ]:
import osmnx
osmnx.geocoder.geocode_to_gdf("Ontdek", which_result=1)


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,POINT (5.06503 51.52289),51.522936,51.522835,5.065082,5.064982,125482119,node,2833894024,51.522886,5.065032,office,accountant,30,0.00001,office,Ontdek Kracht,"Ontdek Kracht, 2a, Dorpsstraat, Wildackers, Go..."


In [ ]:
len("asdf".split("-"))

1

In [ ]:
def merge_close_points(df_points_interest, R=0.001):
    # Calculate the distance between each point and each centroid
    centroids = df_points_interest[["latitude", "longitude"]].to_numpy()
    
    # Calculate the pairwise distances between all centroids
    distances_centroids = squareform(pdist(centroids))

    # Initialize a list to hold the new centroids
    new_centroids = []
    
    # Initialize a set to keep track of used centroids
    used_centroids = set()

    # For each centroid
    for i in range(len(centroids)):
        # Skip this centroid if it has already been used
        if tuple(centroids[i]) in used_centroids:
            continue

        # Find the other centroids that are closer than R distance
        close_centroids_indices = np.where(distances_centroids[i] <= 0.005)[0]

        # Mark these centroids as used
        for index in close_centroids_indices:
            used_centroids.add(tuple(centroids[index]))

        # Calculate the mean latitude and longitude of these close centroids
        close_centroids = centroids[close_centroids_indices]
        mean_centroid = close_centroids.mean(axis=0)
        print(close_centroids)
        print(mean_centroid)
        print("*********************************")

        # Add the new centroid to the list
        new_centroids.append(mean_centroid)

    # Convert the list to a numpy array
    centroids_arr = np.array(new_centroids)
    
    return pd.DataFrame(centroids_arr, columns=["latitude", "longitude"])

In [ ]:
merge_close_points(dfx)

[[50.8388337   4.37163333]]
[50.8388337   4.37163333]
*********************************
[[50.85160807  4.45458292]
 [50.85127281  4.45497492]
 [50.85126268  4.4530968 ]
 [50.85099387  4.45426566]
 [50.85103409  4.45601582]
 [50.85129809  4.45550451]
 [50.85139873  4.45404954]
 [50.85192616  4.45350163]
 [50.85190796  4.45575877]
 [50.84925629  4.45244264]
 [50.85034065  4.4534483 ]
 [50.84976332  4.45308366]
 [50.85061904  4.45298553]
 [50.85220529  4.45375797]
 [50.85161173  4.45344632]
 [50.84965214  4.45205652]
 [50.85251054  4.45386123]
 [50.85091917  4.45511078]
 [50.8499707   4.45272183]
 [50.85049015  4.45407441]]
[50.85100207  4.45393699]
*********************************
[[50.8505712   4.36389656]
 [50.85290571  4.36096842]]
[50.85173846  4.36243249]
*********************************


/data/pgcasado/envs/acute/lib/python3.12/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


,latitude,longitude
0,50.838834,4.371633
1,50.851002,4.453937
2,50.851738,4.362432


In [ ]:
# Calculate the distance between each point and each centroid
centroids = dfx[["latitude", "longitude"]].to_numpy()

# Calculate the pairwise distances between all centroids
distances_centroids = squareform(pdist(centroids))

# Initialize a list to hold the new centroids
new_centroids = []

# Initialize a set to keep track of used centroids
used_centroids = set()

# For each centroid
for i in range(len(centroids)):
    # Skip this centroid if it has already been used
    if tuple(centroids[i]) in used_centroids:
        continue

    # Find the other centroids that are closer than R distance
    close_centroids_indices = np.where(distances_centroids[i] <= 0.005)[0]

    # Mark these centroids as used
    for index in close_centroids_indices:
        used_centroids.add(tuple(centroids[index]))

    # Calculate the mean latitude and longitude of these close centroids
    close_centroids = centroids[close_centroids_indices]
    mean_centroid = close_centroids.mean(axis=0)
    print(close_centroids)
    print(mean_centroid)
    print("*********************************")

    # Add the new centroid to the list
    new_centroids.append(mean_centroid)

# Convert the list to a numpy array
new_centroids = np.array(new_centroids)

# Convert the list to a numpy array
centroids2 = np.array(new_centroids)
dftest = pd.DataFrame(centroids2, columns=["latitude", "longitude"]).drop_duplicates()
print(dfx.shape, dftest.shape)

[[50.8388337   4.37163333]]
[50.8388337   4.37163333]
*********************************
[[50.85160807  4.45458292]
 [50.85127281  4.45497492]
 [50.85126268  4.4530968 ]
 [50.85099387  4.45426566]
 [50.85103409  4.45601582]
 [50.85129809  4.45550451]
 [50.85139873  4.45404954]
 [50.85192616  4.45350163]
 [50.85190796  4.45575877]
 [50.84925629  4.45244264]
 [50.85034065  4.4534483 ]
 [50.84976332  4.45308366]
 [50.85061904  4.45298553]
 [50.85220529  4.45375797]
 [50.85161173  4.45344632]
 [50.84965214  4.45205652]
 [50.85251054  4.45386123]
 [50.85091917  4.45511078]
 [50.8499707   4.45272183]
 [50.85049015  4.45407441]]
[50.85100207  4.45393699]
*********************************
[[50.8505712   4.36389656]
 [50.85290571  4.36096842]]
[50.85173846  4.36243249]
*********************************
(23, 30) (3, 2)


/data/pgcasado/envs/acute/lib/python3.12/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


In [4]:
import warnings
import pandas as pd
from pathlib import Path
data_dir = Path("/data/pgcasado/datasets/acute/")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )

ValueError: No objects to concatenate

In [9]:
dbscan = HDBSCAN(min_samples=3, cluster_selection_epsilon=0.1)

dbscan.fit([[1 ,2],[3,1], [3, 3]])
type(dbscan.labels_)

numpy.ndarray

In [14]:
if a:
    print("A")
else:
    print("B")

NameError: name 'a' is not defined

In [12]:
type(pd.DataFrame([[1 ,2],[3,1], [3, 3]]).values)

numpy.ndarray

In [3]:

def plot_clusters(cluster_mode, altitude_limit, eps, min_samples, selected_tags):
    
    global df  # Use the global df inside the function
    
    # Filter by altitude
    mask_alt = (df["altitude"] > altitude_limit)
    df_filtered_mask = df.loc[mask_alt].copy()
    
    # Define your latitude and longitude limits
    lat_min, lat_max = df_filtered_mask["latitude"].min(), df_filtered_mask["latitude"].max()
    lon_min, lon_max = df_filtered_mask["longitude"].min(), df_filtered_mask["longitude"].max()

    # Create a square polygon
    polygon = Polygon([(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)])

    # Create the dataframe of the place of interest
    #tags = {"building": ["church", "palace", "chapel", "university", "hospital", "government", "stadium", "historic"], "amenities"}
     # Convert selected_tags to a dictionary
    #tags = {tag.split(": ")[0]: [tag.split(": ")[1]] for tag in selected_tags}
    tags = {"building":["university", "chapel"]}
    dfx = ox.features.features_from_polygon(polygon, tags= tags).copy()  # Create a copy to avoid SettingWithCopyWarning
    dfx.dropna(subset=['name'], inplace=True)
    print(dfx.shape)
    dfx['longitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
    dfx['latitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])

    if not cluster_mode:
        warnings.filterwarnings("ignore")
        # Create a DBSCAN object with the updated parameters
        #dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan = HDBSCAN(min_samples=min_samples, cluster_selection_epsilon=eps)

        # Fit the model to the data
        dbscan.fit(df_filtered_mask[['latitude', 'longitude']])
        df_filtered_mask["cluster"] = dbscan.labels_

        dfn = df_filtered_mask[df_filtered_mask["cluster"] >= 0] # cluster -1 is those points with no cluster
        labels = np.delete(dbscan.labels_, np.where(dbscan.labels_ == -1))
    else:
        # Calculate the distance between each point and each centroid
        centroids = dfx[["latitude", "longitude"]].to_numpy()
        points = df_filtered_mask[["latitude", "longitude"]].to_numpy()
        distances = cdist(points, centroids, "euclidean")

        # Find the index of the closest centroid for each point
        clusters = np.argmin(distances, axis=1)

        # Count the number of points in each cluster
        counts = np.bincount(clusters)

        # Identify clusters with more than N points
        N = 100
        large_clusters = np.where(counts > N)[0]

        # Filter out small clusters and their centroids
        filtered_centroids = centroids[large_clusters]

        # Recalculate the distances
        distances = cdist(points, filtered_centroids, "euclidean")
        
        # Find the index of the closest centroid for each point
        clusters = np.argmin(distances, axis=1)
        df_filtered_mask["cluster"] = clusters
        dfn = df_filtered_mask[df_filtered_mask["cluster"] >= 0]

    # Assuming df1 and df2 are your dataframes
    fig = px.scatter_mapbox(dfn, lat='latitude', lon='longitude', color='cluster', size = "altitude",
                            color_continuous_scale=px.colors.cyclical.HSV,
                        zoom=10, height=1000, width=1500,
                        title='Drone Detected Position',)

    # Add scatter_mapbox trace for df2
    if not cluster_mode:
        fig.add_trace(go.Scattermapbox(
            lat=dfx['latitude'],
            lon=dfx['longitude'],
            mode='markers',
            marker=dict(size=10, color='white'),
            name=""
        ))
    else:
        fig.add_trace(go.Scattermapbox(
            lat=dfx['latitude'].iloc[large_clusters],
            lon=dfx['longitude'].iloc[large_clusters],
            mode='markers',
            marker=dict(size=10, color='white'),
            name=""
        ))

    fig.update_layout(
        mapbox_style="dark",
        mapbox_accesstoken=token,
        margin={"r":0,"t":0,"l":0,"b":0},
        autosize=True,
        hovermode='closest',
        showlegend=True
    )
    
    cluster_df = pd.DataFrame()
    for i in range(df_filtered_mask["cluster"].max()):
        df_i = df_filtered_mask[df_filtered_mask["cluster"]==i]
        cluster_df.loc[i, "nº detection"] = df_i.shape[0]
        cluster_df.loc[i, "max_altitude"] = df_i["altitude"].max()
        cluster_df.loc[i, "std_altitude"] = round(df_i["altitude"].std(), 2)
        cluster_df.loc[i, "mean_altitude"] = round(df_i["altitude"].mean(), 2)
        cluster_df.loc[i, "max_distance"] = round(df_i["distance"].max(),2)
        cluster_df.loc[i, "std_distance"] = round(df_i["distance"].std(),2)
        cluster_df.loc[i, "mean_distance"] = round(df_i["distance"].mean(),2)
        #cluster_df.loc[i, "main model"] = df_i["model"].value_counts().idxmax()
        #cluster_df.loc[i, "nº model"] = len(df_i["model"].unique())
        cluster_df.loc[i, "nº ID"] = len(df_i["ident"].unique())
        cluster_df.loc[i, "nº days"] = pd.to_datetime(df_i['timestamp']).dt.date.nunique()

    cluster_df["nº detection"] = cluster_df["nº detection"].astype("int")
    cluster_df["nº ID"] = cluster_df["nº ID"].astype("int")
    cluster_df["nº days"] = cluster_df["nº days"].astype("int")
    #cluster_df["nº model"] = cluster_df["nº model"].astype("int")
    
    display(cluster_df)
    
    fig.show()

# Use interact to create interactive sliders
# Use interact to create interactive sliders
plot = interact(
    plot_clusters,
    cluster_mode=widgets.Checkbox(value=False, description='POI'),
    altitude_limit=widgets.FloatSlider(min=0, max=500, step=10, value=0, description='Altitude Limit'),
    eps=widgets.FloatSlider(min=0.001, max=0.2, step=0.001, value=0.025, description='Eps'),
    min_samples=widgets.IntSlider(min=1, max=50, step=1, value=17, description='Min Samples'),
    selected_tags=widgets.SelectMultiple(options=["building: church", "building: chapel", "building: university", "building: hospital", "building: government", "building: stadium", "amenities: prison", "amenities: police"],
                                         value=["building: university",], description='Tags')
)

NameError: name 'interact' is not defined

In [ ]:
# next step is to consider centroids which are closer than X distance to be consider the same.

data_dir = Path("/data/pgcasado/datasets/acute/")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    df2 = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )
    
mask = (df2['timestamp'] > '2023-06-21') & (df2['timestamp'] <= '2023-12-31') & (df2["station_name"] == "0QRDKC2R03J32P")
df2 = df2[mask] # Create a copy to avoid SettingWithCopyWarning

altitude_limit = 50
mask_alt = (df2["altitude"] > altitude_limit)
df2 = df2.loc[mask_alt]

# Define your latitude and longitude limits
lat_min, lat_max = df2["latitude"].min(), df2["latitude"].max()
lon_min, lon_max = df2["longitude"].min(), df2["longitude"].max()

# Create a square polygon
polygon = Polygon([(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)])

# Create the dataframe of the place of interest
#tags = {"building": ["church", "palace", "chapel", "university", "hospital", "government", "stadium", "historic"]}
tags = {"building":["university",]}
dfx = ox.features.features_from_polygon(polygon, tags=tags)

dfx.dropna(subset=['name'], inplace=True)
dfx['longitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
dfx['latitude']  = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])

# read the shapefile for the world map
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# plot the world map
#world.plot(color='white', edgecolor='black')

# Calculate the distance between each point and each centroid
centroids = dfx[["latitude", "longitude"]].to_numpy()

# Calculate the pairwise distances between all centroids
distances_centroids = squareform(pdist(centroids))

# Define your distance threshold R
R = 0.007  # replace with your actual value

# Initialize a list to hold the new centroids
new_centroids = []

# For each centroid
for i in range(len(centroids)):
    # Find the other centroids that are closer than R distance
    close_centroids = centroids[distances_centroids[i] <= R]
    
    # Calculate the mean latitude and longitude of these close centroids
    mean_centroid = close_centroids.mean(axis=0)
    
    # Add the new centroid to the list
    new_centroids.append(mean_centroid)

# Convert the list to a numpy array
centroids = np.array(new_centroids)

points = df2[["latitude", "longitude"]].to_numpy()
distances0 = cdist(points, centroids, "euclidean")

# Find the index of the closest centroid for each point
clusters0 = np.argmin(distances0, axis=1)

# Find the minimum distance for each point to its closest centroid and keep those with a closer distance than X
X = 0.1
min_distances = np.min(distances0, axis=1)
dist_mask = min_distances <= X
points = points[dist_mask]
clusters0 = clusters0[dist_mask]
df2 = df2[dist_mask]

# Count the number of points in each cluster
counts = np.bincount(clusters0)

# Identify clusters with more than N points
N = 30
large_clusters = np.where(counts > N)[0]

# Filter out small clusters and their centroids
filtered_centroids = centroids[large_clusters]

# Recalculate the distances
distances = cdist(points, filtered_centroids, "euclidean")

# Find the index of the closest centroid for each point
clusters = np.argmin(distances, axis=1)

# Find the minimum distance for each point to its closest centroid and keep those with a closer distance than X
min_distances = np.min(distances, axis=1)
dist_mask = min_distances <= X
points = points[dist_mask]
clusters = clusters[dist_mask]
df2 = df2[dist_mask]
df2["clusters"] = clusters

# plot the clusters
plt.rcParams['figure.figsize'] = [12, 10]
plt.scatter(df2['longitude'], df2['latitude'], c=clusters, cmap = "tab20", marker=".", s=20)
plt.scatter(dfx['longitude'].iloc[large_clusters], dfx['latitude'].iloc[large_clusters], color = "black", marker=".", s=80)

# set the x and y limits of the plot
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.xlim(df2["longitude"].min()-0.01, df2["longitude"].max()+0.01)
plt.ylim(df2["latitude"].min()-0.01, df2["latitude"].max()+0.01)


KeyboardInterrupt: 

In [ ]:
df2['longitude'][dist_mask]

/data/pgcasado/envs/acute/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning:

ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead

/data/pgcasado/envs/acute/lib/python3.12/ast.py:587: DeprecationWarning:

Attribute s is deprecated and will be removed in Python 3.14; use value instead



IndexError: Boolean index has wrong length: 55768 instead of 55758

In [ ]:
import streamlit as st
import pandas as pd
from scipy.spatial.distance import cdist
from shapely.geometry import Polygon
import osmnx as ox
from sklearn.cluster import DBSCAN
import plotly.express as px
import plotly.graph_objects as go

# Define your function
def plot_clusters(cluster_mode, altitude_limit, eps, min_samples, selected_tags):  
    # Use the global df inside the function
    global df  
    mask_alt = (df["altitude"] > altitude_limit)
    df_filtered_mask = df.loc[mask_alt].copy()
    lat_min, lat_max = df_filtered_mask["latitude"].min(), df_filtered_mask["latitude"].max()
    lon_min, lon_max = df_filtered_mask["longitude"].min(), df_filtered_mask["longitude"].max()
    polygon = Polygon([(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)])
    tags = {"building":["university",]}
    dfx = ox.features.features_from_polygon(polygon, tags= tags).copy()  # Create a copy to avoid SettingWithCopyWarning
    dfx.dropna(subset=['name'], inplace=True)
    dfx['longitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
    dfx['latitude'] = dfx['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])
    if not cluster_mode:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        # Fit the model to the data
        dbscan.fit(df_filtered_mask[['latitude', 'longitude']])
        df_filtered_mask["cluster"] = dbscan.labels_
        dfn = df_filtered_mask[df_filtered_mask["cluster"] >= 0] # cluster -1 is those points with no cluster
        labels = np.delete(dbscan.labels_, np.where(dbscan.labels_ == -1))
    else:
        centroids = dfx[["latitude", "longitude"]].to_numpy()
        points = df_filtered_mask[["latitude", "longitude"]].to_numpy()
        distances = cdist(points, centroids, "euclidean")
        clusters = np.argmin(distances, axis=1)
        counts = np.bincount(clusters)
        N = 100
        large_clusters = np.where(counts > N)[0]
        # Filter out small clusters and their centroids
        filtered_centroids = centroids[large_clusters]
        distances = cdist(points, filtered_centroids, "euclidean")
        # Find the index of the closest centroid for each point
        clusters = np.argmin(distances, axis=1)
        df_filtered_mask["cluster"] = clusters
        dfn = df_filtered_mask[df_filtered_mask["cluster"] >= 0]
    fig = px.scatter_mapbox(dfn, lat='latitude', lon='longitude', color='cluster',
                            color_continuous_scale=px.colors.cyclical.HSV,
                        zoom=10, height=1000, width=1500,
                        title='Drone Detected Position',)
    if not cluster_mode:
        fig.add_trace(go.Scattermapbox(
            lat=dfx['latitude'],
            lon=dfx['longitude'],
            mode='markers',
            marker=dict(size=10, color='white'),
            name=""
        ))
    else:
        fig.add_trace(go.Scattermapbox(
            lat=dfx['latitude'].iloc[large_clusters],
            lon=dfx['longitude'].iloc[large_clusters],
            mode='markers',
            marker=dict(size=10, color='white'),
            name=""
        ))
    cluster_df = pd.DataFrame()
    for i in range(df_filtered_mask["cluster"].max()):
        df_i = df_filtered_mask[df_filtered_mask["cluster"]==i]
        cluster_df.loc[i, "nº detection"] = df_i.shape[0]
    cluster_df["nº detection"] = cluster_df["nº detection"].astype("int")
    st.write(cluster_df)
    st.plotly_chart(fig)

# Use Streamlit widgets to get user input
cluster_mode = st.checkbox('POI')
altitude_limit = st.slider('Altitude Limit', min_value=0, max_value=500, value=0, step=10)
eps = st.slider('Eps', min_value=0.001, max_value=0.2, value=0.025, step=0.001)
min_samples = st.slider('Min Samples', min_value=1, max_value=50, value=17)
selected_tags = st.multiselect('Tags', options=["building: church", "building: chapel", "building: university", "building: hospital", "building: government", "building: stadium", "amenities: prison", "amenities: police"], default=["building: university"])

# Call the function when the 'Plot Clusters' button is clicked
if st.button('Plot Clusters'):
    plot_clusters(cluster_mode, altitude_limit, eps, min_samples, selected_tags)
